In [124]:
from bs4 import BeautifulSoup as bs              # 데이터파싱 라이브러리
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import json
import platform
import re
import time
import copy

In [125]:
path = 'chromedriver.exe' if (platform.system() == 'Windows') else '/Users/jg/Desktop/develop/DataTeam/DataProcessing/product/crawling/chromedriver';
driver = webdriver.Chrome(path)

base_url = 'http://www.narscosmetics.co.kr/'

In [126]:
def writeJSON(jsonString, output_name='data.json'):
    with open(output_name,'w',encoding='UTF-8') as file:
        file.write(jsonString)

In [133]:
categorys = ["lips", "cheeks", "face", "eyes", "multi-use", "cult-classics", "narsskin", "makeup-removers", "brushes-tools", ]

In [134]:
def getUrlList():
    urlList = []
    for category in categorys:
        #카테고리별로 접속
        driver.get(base_url + category)
        last_height = driver.execute_script("return document.body.scrollHeight")
        #상품 긁기
        while True:
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        html = driver.page_source
        soup = bs(html,'html.parser')
        products = driver.find_elements_by_xpath("//div[@class='product-image']//a[@class='thumb-link']")
        for product in products:
            urlList.append(product.get_attribute("href"))
    return urlList
        
        #다음 페이지가 있으면 다음 페이지로 이동

In [135]:
urlList = getUrlList()

In [130]:
urlList = list(set(urlList))
print(len(urlList))

35


In [131]:
def getItemDetailByUrl(urlList):
    
    result_json = []
    for idx, item in enumerate(urlList):
        driver.get(item)
        html = driver.page_source
        soup = bs(html,'html.parser')
        # dictionary 생성
        result = {'name':'', 'url':'', 'image':'', 'salePrice':'', 'originalPrice':'', 'color':'', 
                   'category':'', 'brand':'나스','volume':''}
        
        result['name'] = soup.find('h1',{'class':'product-name page-title'}).get_text()
        result['url'] = driver.current_url
        images = []
        
        #Color만큼 
        if soup.find('div',{'class':'selectricItems'}) is None:
            #단품
            if soup.find('div',{'id':'thumbnails'}) is None:
                images.append(soup.find('div',{'class':'product-primary-image'}).find('img')['src'])
                result['image'] = images
            else:
                images = driver.find_elements_by_xpath("//div[@id='thumbnails']//li//a")
                result['image'] = [image.get_attribute('href') for image in images]
            result['salePrice'] = soup.find('span',{'class':'price-sales'}).get_text()
            result['originalPrice'] = result['salePrice']
            result['volume'] = "#"
            result['color'] = "#"
            display(result)
            result_json.append(result)
        else:
            
            colors = driver.find_elements_by_xpath("//div[@class='selectricHideSelect']//option")
            htmlList = [color.get_attribute('data-href') for color in colors]
            totalColors = len(colors)
            for html in htmlList:
                driver.get(html)
                html = driver.page_source
                soup = bs(html,'html.parser')
                result['name'] = soup.find('h1',{'class':'product-name page-title'}).get_text()
                result['url'] = driver.current_url
                if soup.find('div',{'id':'thumbnails'}) is None:
                    images.append(soup.find('div',{'class':'product-primary-image'}).find('img')['src'])
                    result['image'] = images
                else:
                    images = driver.find_elements_by_xpath("//div[@id='thumbnails']//li//a")
                    result['image'] = [image.get_attribute('href') for image in images]
                result['salePrice'] = soup.find('span',{'class':'price-sales'}).get_text()
                result['originalPrice'] = result['salePrice']
                result['volume'] = "#"
                result['color'] = soup.find('span',{'class':'colorName'}).get_text()
                display(result)
                result_json.append(result)
        if idx % (len(urlList)//50) == 0:
            print("%3.1f 퍼센트 진행중" % round(idx / len(urlList) * 100))
#             for color in colors:
                
#                 color.click()
#                 time.sleep(2)
#                 html = driver.page_source
#                 soup = bs(html,'html.parser')
#                 result_dict = copy.deepcopy(result)
#                 result_dict['name'] = soup.find('h1',{'class':'product-name page-title'}).get_text()
#                 result_dict['url'] = driver.current_url
#                 result_dict['color'] = color.text
                
                
#                 if soup.find('div',{'id':'thumbnails'}) is None:
#                     images.append(soup.find('div',{'class':'product-primary-image'}).find('img')['src'])
#                     result_dict['image'] = images
#                 else:
#                     images = driver.find_elements_by_xpath("//div[@id='thumbnails']//li//a")
#                     result_dict['image'] = [image.get_attribute('href') for image in images]
#                 result_dict['salePrice'] = soup.find('div',{'id':'product-content'}).find('span',{'class':'price-sales'}).get_text()
#                 result_dict['salePrice'] = result_dict['originalPrice']
#                 result_dict['volume'] = "#"
                
#                 display(result_dict)
#                 result_json.append(result_dict)
        
    return result_json

In [132]:
result_json = getItemDetailByUrl(urlList)

<selenium.webdriver.remote.webelement.WebElement (session="ba51fcd1a1bc41dec611cca0306e5511", element="0.9803823357552255-3")>

{'name': '파워매트 립 피그먼트 - 아메리칸 우먼',
 'url': 'http://www.narscosmetics.co.kr/on/demandware.store/Sites-KR-Site/en_KR/Product-Variation?pid=999NAC0000053&dwvar_999NAC0000053_color=7845027720',
 'image': ['http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwba0ddabc/hi-res/0607845027720.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwe4a213e0/swatch/0607845027720.jpg'],
 'salePrice': '₩ 37,000',
 'originalPrice': '₩ 37,000',
 'color': '아메리칸 우먼',
 'category': '',
 'brand': '나스',
 'volume': '#'}

<selenium.webdriver.remote.webelement.WebElement (session="ba51fcd1a1bc41dec611cca0306e5511", element="0.11696008367089394-3")>

{'name': '파워매트 립 피그먼트 - 콜 미',
 'url': 'http://www.narscosmetics.co.kr/on/demandware.store/Sites-KR-Site/en_KR/Product-Variation?pid=999NAC0000053&dwvar_999NAC0000053_color=7845027843',
 'image': ['http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwfadff468/hi-res/0607845027843.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw94ee6748/swatch/0607845027843.jpg'],
 'salePrice': '₩ 37,000',
 'originalPrice': '₩ 37,000',
 'color': '콜 미',
 'category': '',
 'brand': '나스',
 'volume': '#'}

<selenium.webdriver.remote.webelement.WebElement (session="ba51fcd1a1bc41dec611cca0306e5511", element="0.041000347307711404-3")>

...

{'name': '립스틱 - 부두 릴리',
 'url': 'http://www.narscosmetics.co.kr/on/demandware.store/Sites-KR-Site/en_KR/Product-Variation?pid=999NAC0000067&dwvar_999NAC0000067_color=7845094128',
 'image': ['http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw7bb29523/hi-res/0607845094128.png',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw171a3f3a/alt1/erdem1.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw819233a3/alt3/0607845094128.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwad74a757/alt4/0607845094128.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/Sites-KR-Site/-/en_KR/v1546225498151/images/noimageshadesmall.png'],
 'salePrice': '₩ 37,000',
 'originalPrice': '₩ 37,000',
 'color': '부두 릴리',
 'category': '',
 'brand': '나스',
 'volume': '#'}

<selenium.webdriver.remote.webelement.WebElement (session="ba51fcd1a1bc41dec611cca0306e5511", element="0.3283903469944329-6")>

{'name': '립스틱 - 와일드 플라워',
 'url': 'http://www.narscosmetics.co.kr/on/demandware.store/Sites-KR-Site/en_KR/Product-Variation?pid=999NAC0000067&dwvar_999NAC0000067_color=7845094135',
 'image': ['http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwf2b43159/hi-res/0607845094135.png',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw171a3f3a/alt1/erdem1.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw819233a3/alt3/0607845094135.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwad74a757/alt4/0607845094135.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/Sites-KR-Site/-/en_KR/v1546225498151/images/noimageshadesmall.png'],
 'salePrice': '₩ 37,000',
 'originalPrice': '₩ 37,000',
 'color': '와일드 플라워',
 'category': '',
 'brand': '나스',
 'volume': '#'}

{'name': '립 글로스 세트',
 'url': 'http://www.narscosmetics.co.kr/defiant-lip-gloss-set/0607845083856.html',
 'image': ['http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw174c6016/hi-res/0607845083856.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwbc6efafe/hi-res/0607845083856_2.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw29bac1a4/hi-res/0607845083856_3.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw9aa6678b/alt1/0607845083856.jpg',
  'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dw4cbf8450/alt2/0607845083856.jpg'],
 'salePrice': '₩ 79,000',
 'originalPrice': '₩ 79,000',
 'color': '#',
 'category': '',
 'brand': '나스',
 'volume': '#'}

[{'name': '파워매트 립 피그먼트 - 익스플리시트 레드',
  'url': 'http://www.narscosmetics.co.kr/on/demandware.store/Sites-KR-Site/en_KR/Product-Variation?pid=999NAC0000053&dwvar_999NAC0000053_color=Explicit%20Red',
  'image': ['http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwb9faebe2/hi-res/607845027379.jpg',
   'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwc69bda15/hi-res/607845027379_v2.jpg'],
  'salePrice': '₩ 37,000',
  'originalPrice': '₩ 37,000',
  'color': '익스플리시트 레드',
  'category': '',
  'brand': '나스',
  'volume': '#'},
 {'name': '파워매트 립 피그먼트 - 익스플리시트 레드',
  'url': 'http://www.narscosmetics.co.kr/on/demandware.store/Sites-KR-Site/en_KR/Product-Variation?pid=999NAC0000053&dwvar_999NAC0000053_color=Explicit%20Red',
  'image': ['http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NARS/default/dwb9faebe2/hi-res/607845027379.jpg',
   'http://en.narscosmetics.co.kr/on/demandware.static/-/Sites-itemmaster_NAR

In [ ]:
output = json.dumps(result_json,ensure_ascii=False, indent='\t')

writeJSON(output, output_name = 'nars.json')